# Customer

In [1]:
import pandas as pd

## Database connection

In [2]:
import yaml
from sqlalchemy import create_engine


with open("../config.yml", "r") as file:
	config = yaml.safe_load(file)
	config_OLTP = config["OLTP"]
	config_OLAP = config["OLAP"]


url_OLTP = (f"{config_OLTP['drivername']}://{config_OLTP['user']}:{config_OLTP['password']}"
			f"@{config_OLTP['host']}:{config_OLTP['port']}/{config_OLTP['database_name']}")

url_OLAP = (f"{config_OLAP['drivername']}://{config_OLAP['user']}:{config_OLAP['password']}"
			f"@{config_OLAP['host']}:{config_OLAP['port']}/{config_OLAP['database_name']}")

OLTP_connection = create_engine(url_OLTP)
OLAP_connection = create_engine(url_OLAP)

## Extraction

In [3]:
customer_dimension = pd.read_sql_table("cliente", OLTP_connection)
city = pd.read_sql_table("ciudad", OLTP_connection)

## Transformation

In [6]:
customer_dimension = customer_dimension.merge(city, on="ciudad_id", how="left")

In [7]:
customer_dimension.rename(
	columns={
		"cliente_id": "original_customer_id",
		"nombre_y": "customer_city",
	}, inplace=True
)

In [9]:
customer_dimension = customer_dimension[["original_customer_id", "customer_city"]]

In [10]:
customer_dimension.reset_index(inplace=True)
customer_dimension.rename(columns={ "index": "customer_id" }, inplace=True)
customer_dimension.set_index("customer_id", inplace=True)

## Load

In [12]:
customer_dimension.to_sql(
	"CUSTOMER_DIMENSION", OLAP_connection, if_exists="replace", index=True
)

27